In [1]:
!pip3 install pyspark


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("KaggleParticipation").getOrCreate()


25/05/18 20:51:54 WARN Utils: Your hostname, LT-W-7826.local resolves to a loopback address: 127.0.0.1; using 192.168.31.254 instead (on interface en0)
25/05/18 20:51:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/18 20:51:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
users = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/users.csv",
    header=True,
    inferSchema=True,
)
competitions = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/competitions.csv",
    header=True,
    inferSchema=True,
)
teams = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/teams.csv",
    header=True,
    inferSchema=True,
)
teammemberships = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/teammemberships.csv",
    header=True,
    inferSchema=True,
)
submissions = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/submissions.csv",
    header=True,
    inferSchema=True,
)


In [3]:
from pyspark.sql.functions import col

submissions_df = submissions.alias("s")
teams_df = teams.alias("t")
users_df = users.alias("u")


In [4]:
competitions_df = (
    spark.read.option("header", True)
    .option("inferSchema", True)
    .option("mode", "DROPMALFORMED")
    .csv(
        "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/competitions.csv"
    )
    .alias("c")
)

competitions_df = competitions_df.filter(
    col("Id").isNotNull() & col("Slug").isNotNull() & col("DeadlineDate").isNotNull()
)


In [7]:
competitions_df.head(5)

25/05/18 20:45:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(Id='<pre>ID', Slug='class1', Title='class2', Subtitle='class3', HostSegmentTitle='class4', ForumId='class5', OrganizationId='class6', EnabledDate='class7', DeadlineDate='class8', ProhibitNewEntrantsDeadlineDate='class9<br />0', TeamMergerDeadlineDate='0.1', TeamModelDeadlineDate='0.7', ModelSubmissionDeadlineDate='0.05', FinalLeaderboardHasBeenVerified='0.05', HasKernels='0.1', OnlyAllowKernelSubmissions='0', HasLeaderboard='0', LeaderboardPercentage='0', ScoreTruncationNumDecimals='0<br />1', EvaluationAlgorithmAbbreviation='0.7', EvaluationAlgorithmName='0.1', EvaluationAlgorithmDescription='0.05', EvaluationAlgorithmIsMax='0.05', MaxDailySubmissions='0.1', NumScoredSubmissions='0', MaxTeamSize='0', BanTeamMergers='0', EnableTeamModels='0<br />2', RewardType='0.05', RewardQuantity='0.05', NumPrizes='0.1', UserRankMultiplier='0.7', CanQualifyTiers='0.1', TotalTeams='0', TotalCompetitors='0', TotalSubmissions='0', LicenseName='0<br />3', Overview='0', Rules='0', DatasetDescription

In [5]:
sub_with_team = submissions_df.join(teams_df, col("s.TeamId") == col("t.Id"), "left")

sub_with_team_comp = sub_with_team.join(
    competitions_df, col("t.CompetitionId") == col("c.Id"), "left"
)

sub_with_user = sub_with_team_comp.join(
    users_df, col("s.SubmittedUserId") == col("u.Id"), "left"
)


In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

user_dim = (
    users_df.select("Id", "UserName", "Country", "PerformanceTier")
    .dropDuplicates()
    .withColumn("UserKey", row_number().over(Window.orderBy("Id")))
    .alias("ud")
)

team_dim = (
    teams_df.select("Id", "TeamName", "IsBenchmark")
    .dropDuplicates()
    .withColumn("TeamKey", row_number().over(Window.orderBy("Id")))
    .alias("td")
)

competition_dim = (
    competitions_df.select("Id", "Slug", "DeadlineDate")
    .dropDuplicates()
    .withColumn("CompetitionKey", row_number().over(Window.orderBy("Id")))
    .alias("cd")
)


In [7]:
from pyspark.sql.functions import to_date, dayofmonth, month, year, hour, date_format

time_dim = (
    submissions_df.select("SubmissionDate")
    .dropDuplicates()
    .withColumn("TimeKey", to_date("SubmissionDate", "MM/dd/yyyy"))
    .filter(col("TimeKey").isNotNull())
    .withColumn("Day", dayofmonth("TimeKey"))
    .withColumn("Month", month("TimeKey"))
    .withColumn("Year", year("TimeKey"))
    .withColumn(
        "Hour", hour("SubmissionDate")
    )
    .withColumn("DayOfWeek", date_format("TimeKey", "EEEE"))
    .select("TimeKey", "Day", "Month", "Year", "Hour", "DayOfWeek")
    .alias("tdim")
)


In [8]:
facts = (
    sub_with_user.join(user_dim, col("s.SubmittedUserId") == col("ud.Id"), "left")
    .join(team_dim, col("s.TeamId") == col("td.Id"), "left")
    .join(competition_dim, col("t.CompetitionId") == col("cd.Id"), "left")
    .join(time_dim, to_date(col("s.SubmissionDate")) == col("tdim.TimeKey"), "left")
    .select(
        col("s.Id").alias("SubmissionId"),
        col("tdim.TimeKey"),
        col("ud.UserKey"),
        col("td.TeamKey"),
        col("cd.CompetitionKey"),
        col("s.PublicScoreLeaderboardDisplay"),
        col("s.PrivateScoreLeaderboardDisplay"),
        col("s.IsAfterDeadline"),
    )
)

facts = facts.repartition(10)


In [9]:
# user_dim.write.json(
#     "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/UserDim"
# )
# team_dim.write.json(
#     "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/TeamDim"
# )
# competition_dim.write.json(
#     "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/CompetitionDim"
# )
# time_dim.write.json(
#     "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/TimeDim"
# )
facts.write.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/SubmissionFact"
)


25/05/18 20:52:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 20:52:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 20:52:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 20:52:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 20:52:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 20:52:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 2

Py4JJavaError: An error occurred while calling o199.json.
: org.apache.spark.SparkException: Job 28 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1253)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1251)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:1251)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:3087)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$stop$3(DAGScheduler.scala:2973)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2973)
	at org.apache.spark.SparkContext.$anonfun$stop$12(SparkContext.scala:2258)
	at org.apache.spa

25/05/18 20:54:52 ERROR Utils: Uncaught exception in thread Executor task launch worker for task 2.0 in stage 47.0 (TID 235)
java.lang.NullPointerException
	at org.apache.spark.scheduler.Task.$anonfun$run$3(Task.scala:146)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.Task.run(Task.scala:144)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:94)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:623)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
25/05/18 20:54:52 ERROR Executor: Exce